# Hello!
This is a walkthrough to train the University of Waterloo's submission model.

To find all our experiments and code, see our original [repo](https://github.com/trevor-yu-087/climatehack.ai-2024), but beware, it is not documented, or well organized.

# Environment set up

We use docker to package dependencies. If you are using VScode or a JetBrains IDE, the devcontainers extension should be able to use the .devcontainer directory to build the docker image and use it as a development environment.

If you do not want to use docker, you can (hopefully) get set up by running:

- `pip install -r local-requirements.txt`
- `conda install cartopy`
- `pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121`

A machine with a CUDA enabled GPU is required.

# Download data
Our model used pv, hrv and weather data.

For this example we'll only be downloading a few months of data

In [ ]:
import huggingface_hub
from os import makedirs

datadir = "/workspaces/waterloo-climatehack/data" # change this
makedirs(datadir, exist_ok=True)

huggingface_hub.snapshot_download(
    repo_id="climatehackai/climatehackai-2023", 
    local_dir=datadir, 
    cache_dir=datadir + '/cache',
    local_dir_use_symlinks=False, 
    repo_type="dataset",
    ignore_patterns=["aerosols/*", "satellite-nonhrv/*"],
    allow_patterns=["*10.zarr.zip", "*11.zarr.zip", "*10.parquet", "*11.parquet"]
)

# Generating PV Site Features
We generate site specific features (such the site's max and average output during each month).

# Data Loading
Now let's run our dataset class to validate that our data is set up correctly.

In [4]:
from dataset import get_datasets
import yaml

In [9]:
# yaml file that is used to configure training runs
CONFIG_FILE_NAME = "train.yaml"

with open(CONFIG_FILE_NAME) as file:
    config = yaml.safe_load(file)

In [10]:
train_ds, test_ds = get_datasets(
    config["data_path"],
    (config["start_date"], config["end_date"]),
    batch_size=config["batch_size"],
    hrv="hrv" in config["modalities"],
    weather="weather" in config["modalities"],
    metadata="metadata" in config["modalities"],
    seed=config["seed"],
    pv_features_file=config["pv_features_file"],
    test_size=config["test_size"],
    hrv_crop=config["hrv_crop"],
    weather_crop=config["weather_crop"],
    zipped=config["zipped"],
    offset_start_time=config["offset_start_time"]
)

FileNotFoundError: [Errno 2] No such file or directory: '/root/data/climatehackai/pv/2021/10.parquet'